In [3]:
import zipfile
import os

with zipfile.ZipFile("/Users/samyuktha/Downloads/KL Photos.zip", "r") as zip_ref:
    zip_ref.extractall("destination_folder")

In [ ]:
from geminiai import *

image="IMG_6436.HEIC"

describe_image(dir_path+image)

In [3]:
dir_path = "/Users/samyuktha/Downloads/KL Photos/"

In [ ]:
from preprocess import *

mediaitems = extract_metadata(dir_path)

mediaitems = update_places(mediaitems)

In [2]:
from geminiai import *

import json

with open("mediaitems.json", "r") as json_file:
    mediaitems = json.load(json_file)


mediaitems = generate_descriptions(dir_path, mediaitems)




In [ ]:
mediaitems.sort(key=lambda item: item["datetime"])

mediaitems

In [4]:
import json
from geminiai import *

with open("mediaitems.json", "r") as json_file:
    mediaitems = json.load(json_file)


narration_script = get_script(mediaitems)

narration_script

{'title': 'Malaysian Adventure: From City to Sky',
 'description': 'Come along with us on our exciting trip to Malaysia, where we explored bustling cities, encountered exotic wildlife, and soared to new heights!',
 'scenes': [{'scene_number': 1,
   'media_type': 'video',
   'media_source': 'img_6423.mov',
   'text': "And we're off! Starting our Malaysian adventure at the Chennai airport, super excited for everything in store!"},
  {'scene_number': 2,
   'media_type': 'photo',
   'media_source': 'img_6488.jpeg',
   'text': 'First stop, the aquarium! Getting up close with some amazing underwater creatures. Look at all the colorful fish!'},
  {'scene_number': 3,
   'media_type': 'video',
   'media_source': 'img_6626.mov',
   'text': "Made a new feathered friend at Farm in the City! This little guy's name is Oscar, and he's quite the character."},
  {'scene_number': 4,
   'media_type': 'photo',
   'media_source': 'img_6588.jpeg',
   'text': 'Taking in the beauty and culture at the Batu Cav

In [5]:
def synthesize_text(text,filename):
    """Synthesizes speech from the input string of text."""
    from google.cloud import texttospeech

    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Journey-F",
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE,
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    response = client.synthesize_speech(
        request={"input": input_text, "voice": voice, "audio_config": audio_config}
    )

    # The response's audio_content is binary.
    with open(filename+".mp3", "wb") as out:
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"')

In [7]:
import shutil
from moviepy.editor import (
    VideoFileClip,
    ImageClip,
    concatenate_videoclips,
    TextClip,
    CompositeVideoClip,
    concatenate_audioclips,
    AudioFileClip,
)
from moviepy.video.fx.resize import resize
import os
from moviepy.video import fx as vfx
from moviepy.video.fx import speedx


clips = []

# Define a standard resolution
standard_resolution = (1080, 1920)  # Width x Height


os.makedirs("./audio")

audio_clips = []
# Process each scene
for i in range(len(narration_script["scenes"])):
    scene = narration_script["scenes"][i]

    text = scene["text"]

    filepath = os.path.join(dir_path, scene["media_source"]).lower()

    print(filepath)

    if os.path.exists(filepath):

        if not os.path.exists("./audio/" + str(i) + ".mp3"):

            synthesize_text(text, "./audio/" + str(i))

        audio_clip = AudioFileClip("./audio/" + str(i) + ".mp3")

        duration = audio_clip.duration

        if filepath.endswith((".mp4", ".mov")):
            clip = VideoFileClip(filepath)

            if duration < clip.duration:

                clip = clip.subclip(0, duration)
                
            clip=clip.resize(newsize=(standard_resolution[0], standard_resolution[1]))

        else:
            clip = ImageClip(filepath).set_duration(duration)

        # txt_clip = (
        #     TextClip(
        #         scene["narration"]["text"],
        #         color="white",
        #         font="Arial",
        #         fontsize=50,
        #         method="caption",
        #         stroke_width=2,
        #     )
        #     .set_position("bottom")
        #     .set_duration(float(duration))
        # )
        # clip = CompositeVideoClip([clip, txt_clip])
        clips.append(clip)

        audio_clips.append(audio_clip)


# shutil.rmtree("./audio")
# Concatenate all clips
final_clip = concatenate_videoclips(clips, method="compose")

final_audio_clip = concatenate_audioclips(audio_clips)

final_audio_clip.write_audiofile("final_trip_audio.mp3")

if final_clip.duration != final_audio_clip.duration:

    if final_clip.duration > final_audio_clip.duration:
        # Speed up the video clip to match the audio clip's duration
        speed_factor = final_clip.duration / final_audio_clip.duration
        final_clip = final_clip.fx(vfx.speedx, speed_factor)
    else:
        # Speed up the audio clip to match the video clip's duration

        speed_factor = final_audio_clip.duration / final_clip.duration
        print(speed_factor)
        cmd = (
            'ffmpeg -y -i final_trip_audio.mp3 -filter:a "atempo={}" output.mp3'.format(
                speed_factor
            )
        )

        os.system(cmd)


# Write the result to a file
final_clip.write_videofile("final_trip_video.mp4")

/users/samyuktha/downloads/kl photos/img_6423.mov
Audio content written to file "output.mp3"
/users/samyuktha/downloads/kl photos/img_6488.jpeg
Audio content written to file "output.mp3"
/users/samyuktha/downloads/kl photos/img_6626.mov
Audio content written to file "output.mp3"
/users/samyuktha/downloads/kl photos/img_6588.jpeg
Audio content written to file "output.mp3"
/users/samyuktha/downloads/kl photos/img_6700.mov
Audio content written to file "output.mp3"
/users/samyuktha/downloads/kl photos/img_6709.jpeg
Audio content written to file "output.mp3"
/users/samyuktha/downloads/kl photos/img_6784.mov
Audio content written to file "output.mp3"
/users/samyuktha/downloads/kl photos/img_6884.mov
Audio content written to file "output.mp3"
/users/samyuktha/downloads/kl photos/img_6853.jpeg
Audio content written to file "output.mp3"
/users/samyuktha/downloads/kl photos/img_6893.jpeg
Audio content written to file "output.mp3"
MoviePy - Writing audio in final_trip_audio.mp3


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers      
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

MoviePy - Done.
1.1867957746478874


[out#0/mp3 @ 0x15ae3aca0] video:0KiB audio:888KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.027708%
size=     888KiB time=00:00:56.80 bitrate= 128.1kbits/s speed= 174x    


Moviepy - Building video final_trip_video.mp4.
MoviePy - Writing audio in final_trip_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_trip_video.mp4



Moviepy - Done !
Moviepy - video ready final_trip_video.mp4


In [9]:
cmd = "ffmpeg -y -i final_trip_video.mp4 -i output.mp3 -map 0:v:0 -map 1:a:0 -c:v copy -c:a aac combined_video.mp4"
os.system(cmd)

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino --e

0

In [8]:
def list_voices():
    """Lists the available voices."""
    from google.cloud import texttospeech

    client = texttospeech.TextToSpeechClient()

    # Performs the list voices request
    voices = client.list_voices()

    english_voices = []

    for voice in voices.voices:
        
        for language_code in voice.language_codes:
            if "en" in language_code:

                english_voices.append(voice)

    print(english_voices)

list_voices()

[language_codes: "en-AU"
name: "en-AU-Neural2-A"
ssml_gender: FEMALE
natural_sample_rate_hertz: 24000
, language_codes: "en-AU"
name: "en-AU-Neural2-B"
ssml_gender: MALE
natural_sample_rate_hertz: 24000
, language_codes: "en-AU"
name: "en-AU-Neural2-C"
ssml_gender: FEMALE
natural_sample_rate_hertz: 24000
, language_codes: "en-AU"
name: "en-AU-Neural2-D"
ssml_gender: MALE
natural_sample_rate_hertz: 24000
, language_codes: "en-AU"
name: "en-AU-News-E"
ssml_gender: FEMALE
natural_sample_rate_hertz: 24000
, language_codes: "en-AU"
name: "en-AU-News-F"
ssml_gender: FEMALE
natural_sample_rate_hertz: 24000
, language_codes: "en-AU"
name: "en-AU-News-G"
ssml_gender: MALE
natural_sample_rate_hertz: 24000
, language_codes: "en-AU"
name: "en-AU-Polyglot-1"
ssml_gender: MALE
natural_sample_rate_hertz: 24000
, language_codes: "en-AU"
name: "en-AU-Standard-A"
ssml_gender: FEMALE
natural_sample_rate_hertz: 24000
, language_codes: "en-AU"
name: "en-AU-Standard-B"
ssml_gender: MALE
natural_sample_rate_